In [19]:
from dotenv import load_dotenv
import os
load_dotenv(override=True) # take environment variables from .env.
ai_foundry_endpoint = os.getenv("AI_FOUNDRY_ENDPOINT")
ai_agent_id = os.getenv("AI_AGENT_ID")

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential
from azure.ai.agents.models import ListSortOrder

print("test")

project = AIProjectClient(
    credential=DefaultAzureCredential(),
    endpoint=ai_foundry_endpoint)

agent = project.agents.get_agent(ai_agent_id)

test


In [10]:
thread = project.agents.threads.create()
print(f"Created thread, ID: {thread.id}")

Created thread, ID: thread_1ImGDCxIuQ5ZUVwxLMCS80ch


In [13]:
message = project.agents.messages.create(
    thread_id=thread.id,
    role="user",
    content="How to deploy pods?"
)

In [14]:
run = project.agents.runs.create_and_process(
    thread_id=thread.id,
    agent_id=agent.id)

if run.status == "failed":
    print(f"Run failed: {run.last_error}")
else:
    messages = project.agents.messages.list(thread_id=thread.id, order=ListSortOrder.ASCENDING)

    for message in messages:
        if message.text_messages:
            print(f"{message.role}: {message.text_messages[-1].text.value}")

MessageRole.USER: How to deploy pods?
MessageRole.AGENT: To deploy pods in Kubernetes, you can create a deployment. Here’s an example of how to do it using `kubectl`:

1. **Create a Deployment**:
   You can create a deployment for an application, such as an nginx instance, by using the following command:
   ```shell
   kubectl create deployment --image=nginx nginx-app
   ```
   This command will create a deployment named `nginx-app` that runs nginx.

2. **Expose the Deployment**:
   After creating the deployment, you can expose it using a service:
   ```shell
   kubectl expose deployment nginx-app --port=80 --name=nginx-http
   ```
   This command creates a service named `nginx-http` and exposes port 80.

3. **Run a Pod in the Foreground**:
   If you want to run a pod directly in the foreground, you can use:
   ```shell
   kubectl run [-i] [--tty] --attach <name> --image=<image>
   ```
   Replace `<name>` with your desired pod name and `<image>` with the container image you want to run

In [11]:
def list_all_threads():
    """
    Lists all active conversation threads for the project.
    """
    print("Listing all active threads...")
    try:
        threads = project.agents.threads.list()
        if not threads:
            print("No active threads found.")
        else:
            for thread in threads:
                print(f" - Thread ID: {thread.id}")
        print("-" * 20)
    except Exception as e:
        print(f"An error occurred while listing threads: {e}")

In [12]:
list_all_threads()

Listing all active threads...
 - Thread ID: thread_1ImGDCxIuQ5ZUVwxLMCS80ch
 - Thread ID: thread_5HmPXh1hvHRXzKL4UyH8yzQc
--------------------


In [ ]:
def delete_all_threads():
    print("Deleting all active threads...")
    try:
        threads = project.agents.threads.list()
        if not threads:
            print("No active threads found.")
        else:
            for thread in threads:
                if thread.id != "thread_5HmPXh1hvHRXzKL4UyH8yzQc":
                    project.agents.threads.delete(thread.id)
        print("-" * 20)
    except Exception as e:
        print(f"An error occurred while deleting threads: {e}")

In [8]:
delete_all_threads()

Deleting all active threads...
An error occurred while listing threads: (None) No thread found with id 'thread_uLFEtCxhUDir37sQpz76VI1u'.
Code: None
Message: No thread found with id 'thread_uLFEtCxhUDir37sQpz76VI1u'.


In [5]:
project.agents.threads.delete("thread_h2kDGFJLe5uYmpexzL9gl6Zv")